In [1]:
import pandas as pd
import numpy as np
from tabula import read_pdf
import re
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', None)

## 1. Upload Data Sources

#### Origin: Bussiness Informatics Course Catalog

In [2]:
bi_file = read_pdf('../data/origin/MK_MSc_Wifo_2022_2023_29082022.pdf', pages='all')
bi_file_copy = bi_file.copy()

# Keep tables with information (>2)
wifo_df = []
for table in bi_file_copy:
    if len(table)>2:
        wifo_df.append(table)
len(wifo_df)

Got stderr: Nov 29, 2022 1:48:07 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Nov 29, 2022 1:48:07 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



103

#### Origin: Data Science Course Catalog

In [3]:
data_science_file = read_pdf('../data/origin/MK_MMDS_2022_23_neue_PO_29082022.pdf', pages='all')
data_science_file_copy = data_science_file.copy()

# Keep tables with information (>2)
df_mmds = []
for table in data_science_file:
    if len(table)>2:
        df_mmds.append(table)
len(df_mmds)

26

#### Origin: Data Science List of Courses

In [4]:
courses = pd.read_csv('../data/processed/mmds_courses.csv')
courses

,module_code,Name of Module,Offered,Language,ECTS,moduleCategory
0,CS 450,Programming Course,HWS,E,6,Fundamentals
1,CS 460,Database Technology,FSS,E,6,Fundamentals
2,CS 470,Python for Data Scientists,FSS,E,6,Fundamentals
3,NaN,Multivariate Analyses,HWS,E,6,Fundamentals
4,NaN,Tutorial Multivariate Analyses,HWS,E,2,Fundamentals
5,NaN,Empirische Methoden der Politikwissenschaft,HWS,G/E,6,Fundamentals
6,AC 651,Additional Course – Data Management,HWS/FSS,E,AC 651*,Data Management
7,CS 500,Advanced Software Engineering,HWS,E,6,Data Management
8,CS 530,Database Systems II,FSS,E,6,Data Management
9,CS 550,Algorithmics,FSS/HWS,E,6,Data Management


## 2. Data Preprocessing

### 2.1 Courses Identification

####  MMDS Courses in Bussiness Informatics Catalog

MMDS courses found in BI catalog

In [5]:
# Find the tables from BI where the information about the MMDS courses is located

# Create and index of the found courses and the table where the information is located
mask_na = courses.module_code.notna() # Ignore courses without a code (4 courses)
courses_ = courses[mask_na].copy()
wifo_df_idx = []
for item in courses_.module_code:
    for idx in range(len(wifo_df)):
        if item==wifo_df[idx].columns[0]:
            wifo_df_idx.append([idx,item])
            break
print('Index of MMDS courses found in BI catalog: (%s)\n'%len (wifo_df_idx), wifo_df_idx)
            
# List of the MMDS courses found in BI catalog
def extract_item(lst):
    return [item[1] for item in lst]
matched_courses_wifo = extract_item(wifo_df_idx)

Index of MMDS courses found in BI catalog: (25)
 [[2, 'CS 500'], [4, 'CS 530'], [5, 'CS 550'], [8, 'CS 560'], [18, 'CS 600'], [28, 'CS 662'], [30, 'IE 630'], [32, 'IE 650'], [37, 'IE 663'], [39, 'IE 670'], [58, 'IE 691'], [54, 'IE 683'], [10, 'IE 500'], [12, 'IE 560'], [35, 'IE 661'], [41, 'IE 671'], [43, 'IE 672'], [45, 'IE 675b'], [47, 'IE 676'], [49, 'IE 678'], [56, 'IE 684'], [51, 'IE 692'], [60, 'IE 694'], [62, 'IE 696'], [24, 'CS 652']]


MMDS courses not found in BI catalog

In [6]:
# List of the MMDS courses NOT found in BI catalog
unmatched_course_in_wifo = []
for item in courses_.module_code:
     if item not in matched_courses_wifo:
            unmatched_course_in_wifo.append(item)
            
print('List of MMDS courses NOT found in BI catalog: (%s)\n'%len (unmatched_course_in_wifo), unmatched_course_in_wifo)

List of MMDS courses NOT found in BI catalog: (19)
 ['CS 450', 'CS 460', 'CS 470', 'AC 651', 'IS 540', 'IS 556', 'AC 652', 'DA 110', 'IS 622', 'MAA 519', 'MAB 504', 'MAB 508', 'MAC 404', 'MAC 502', 'MAC 507', 'MAC 527', 'MKT 511', 'MKT 545', 'CS 718']


####  MMDS Courses in MMDS Catalog

MMDS courses found in MMDS catalog

In [7]:
df_mmds_idx = []
for item in unmatched_course_in_wifo:
    for idx in range(len(df_mmds)):
        if item==df_mmds[idx].columns[0]:
            df_mmds_idx.append([idx,item])
            break
print('Index of MMDS courses found in MMDS catalog: (%s)\n'%len (df_mmds_idx), df_mmds_idx)

# List of the MMDS courses found in MMDS catalog
matched_courses_mmds = extract_item(df_mmds_idx)

Index of MMDS courses found in MMDS catalog: (7)
 [[2, 'CS 450'], [4, 'CS 460'], [6, 'CS 470'], [9, 'AC 651'], [12, 'AC 652'], [13, 'DA 110'], [18, 'CS 718']]


MMDS courses not found in BI or MMDS catalog

In [8]:
# List of the MMDS courses NOT found in MMDS catalog
unmatched_course_in_wifo_mmds = []
for item in courses_.module_code:
     if item not in matched_courses_mmds+matched_courses_wifo:
            unmatched_course_in_wifo_mmds.append(item)
            
print('List of MMDS courses NOT found in BI or MMDS catalog: (%s)\n'%len (unmatched_course_in_wifo_mmds), unmatched_course_in_wifo_mmds)

List of MMDS courses NOT found in BI or MMDS catalog: (12)
 ['IS 540', 'IS 556', 'IS 622', 'MAA 519', 'MAB 504', 'MAB 508', 'MAC 404', 'MAC 502', 'MAC 507', 'MAC 527', 'MKT 511', 'MKT 545']


### 2.2 Extraction of Courses Metadata from BI Catalog

#### Tables that are the start of courses metadata
Create list with tables that are the start of courses tables

In [9]:
# List of table indexes that are the start of a metadata tables of courses 
course_idx = []
for idx in range(len(wifo_df)):
    if bool(re.search('^[a-zA-Z]{1,3} ',wifo_df[idx].columns[0])):
        course_idx.append(idx)
print('List of table indexes of BI that are courses: (%s)\n'%len(course_idx), course_idx)

List of table indexes of BI that are courses: (46)
 [2, 4, 5, 8, 10, 12, 18, 20, 22, 24, 26, 28, 30, 32, 35, 37, 39, 41, 43, 45, 47, 49, 51, 54, 56, 58, 60, 62, 64, 68, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101]


Fix format of tables with more than 2 columns

In [10]:
# List of tables with more than 2 columns ('Unnamed')
three_column_courses = []
for item in course_idx:
    if len(wifo_df[item].columns)>2:
        three_column_courses.append(item)
print('List of table indexes with more than 2 columns: \n', three_column_courses)


# Dropping extra columns from the tables that are in three_column_courses
drop_columns = ['Unnamed: 0','Unnamed: 1','Unnamed: 2']
for idx in three_column_courses:
    wifo_df[idx] = wifo_df[idx].loc[:,~wifo_df[idx].columns.isin(drop_columns)].copy()

# wifo_df[5].head() --- to validate if it work

List of table indexes with more than 2 columns: 
 [2, 5, 20, 24, 95, 97]


Delete tables with just 1 column

In [11]:
# List of tables with just 1 column
one_column_courses = []
for item in course_idx:
    if len(wifo_df[item].columns)==1:
        one_column_courses.append(item)
print('List of table indexes with just 1 column: \n', one_column_courses)

# Remove the 22 idx from course index (not a course from mmds)
course_idx.remove(22)

List of table indexes with just 1 column: 
 [22]


Final list with tables that are the start of courses tables

In [12]:
print('List of table indexes of BI that are courses: (%s)\n'%len(course_idx), course_idx)

List of table indexes of BI that are courses: (45)
 [2, 4, 5, 8, 10, 12, 18, 20, 24, 26, 28, 30, 32, 35, 37, 39, 41, 43, 45, 47, 49, 51, 54, 56, 58, 60, 62, 64, 68, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101]


#### Tables that are not the start of courses metadata
Create list with tables that are not the start of courses tables

In [13]:
# List of table indexes that are not the start of a metadata tables of courses 
non_course_idx = []
for idx in range(len(wifo_df)):
    if not bool(re.search('^[a-zA-Z]{1,3} ',wifo_df[idx].columns[0])):
        non_course_idx.append(idx)
print('List of table indexes of BI that are not courses: (%s)\n'%len(non_course_idx), non_course_idx)

List of table indexes of BI that are not courses: (57)
 [0, 1, 3, 6, 7, 9, 11, 13, 14, 15, 16, 17, 19, 21, 23, 25, 27, 29, 31, 33, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 53, 55, 57, 59, 61, 63, 65, 66, 67, 69, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102]


Fix format of tables with more than 2 columns, and delete tables without courses' detail

In [14]:
# List of tables with more than 2 columns ('Unnamed')
three_column_non_courses = []
for item in non_course_idx:
    if len(wifo_df[item].columns)>2:
        three_column_non_courses.append(item)
print('List of table indexes with more than 2 columns: \n', three_column_non_courses)


List of table indexes with more than 2 columns: 
 [0, 1, 6, 11, 14, 15, 16, 17, 19, 25, 33, 38, 50, 59, 61, 65, 66, 67, 72, 74, 80, 82, 84, 94, 96, 100]


In [15]:
# Handling the data frames with more thant 3 columns
remove_non_course_idx = [0,1,14,15,16,17,65,66,67] # remove 0,1,14,15,16,17,65,66,67 indexes
for i in remove_non_course_idx:
    three_column_non_courses.remove(i)
    non_course_idx.remove(i)
    
for idx in three_column_non_courses:
    wifo_df[idx] = wifo_df[idx].loc[:,wifo_df[idx].columns[0:2]].copy()

Check tables with just 1 column

In [16]:
# List of tables with just 1 column
one_column_non_courses = []
for item in non_course_idx:
    if len(wifo_df[item].columns)==1:
        one_column_non_courses.append(item)
print('List of table indexes with just 1 column:\n', one_column_non_courses)


List of table indexes with just 1 column:
 []


Final list with tables that are not the start of courses tables

In [17]:
print('List of table indexes of BI that are not courses: (%s)\n'%len(non_course_idx), non_course_idx)

List of table indexes of BI that are not courses: (48)
 [3, 6, 7, 9, 11, 13, 19, 21, 23, 25, 27, 29, 31, 33, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 53, 55, 57, 59, 61, 63, 69, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102]


### 2.3 Build the Metadata Dataset from MMDS Courses from BI Catalog

Join the frames that contain metadata about the MMDS courses

In [18]:
#Desired output with data frame
final_columns = ['module_code','module_name','property','property_value']
final_wifo_frame = pd.DataFrame(columns=final_columns)

#1st loop to append the consider course frames
for idx in range(len(course_idx)):
    course_frame = wifo_df[course_idx[idx]].copy()
    course_name = wifo_df[course_idx[idx]].columns[1]
    course_code = wifo_df[course_idx[idx]].columns[0]
    course_frame.columns = final_columns[2:]
    print(course_idx[idx])
    #if not last course
    if (idx+1)<len(course_idx):
        #2nd for loop to append the non course frames
        for nc_idx in non_course_idx:
            if (nc_idx>course_idx[idx]) & (nc_idx<course_idx[idx+1]):
                intermediate_frame = wifo_df[nc_idx].copy()
                intermediate_frame.columns = final_columns[2:]
                course_frame = pd.concat([course_frame,intermediate_frame],ignore_index = True)
                print('--',nc_idx)
    
    #for last course    
    else:
        for nc_idx in non_course_idx:
            if (nc_idx>course_idx[idx]):
                intermediate_frame = wifo_df[nc_idx].copy()
                intermediate_frame.columns = final_columns[2:]
                course_frame = pd.concat([course_frame,intermediate_frame],ignore_index = True)
                print('--',nc_idx)
    course_frame['module_code'] = course_code
    course_frame['module_name'] = course_name
    final_wifo_frame = pd.concat([final_wifo_frame,course_frame])
    print('appended')

2
-- 3
appended
4
appended
5
-- 6
-- 7
appended
8
-- 9
appended
10
-- 11
appended
12
-- 13
appended
18
-- 19
appended
20
-- 21
-- 23
appended
24
-- 25
appended
26
-- 27
appended
28
-- 29
appended
30
-- 31
appended
32
-- 33
-- 34
appended
35
-- 36
appended
37
-- 38
appended
39
-- 40
appended
41
-- 42
appended
43
-- 44
appended
45
-- 46
appended
47
-- 48
appended
49
-- 50
appended
51
-- 52
-- 53
appended
54
-- 55
appended
56
-- 57
appended
58
-- 59
appended
60
-- 61
appended
62
-- 63
appended
64
appended
68
-- 69
-- 70
appended
71
-- 72
appended
73
-- 74
appended
75
-- 76
appended
77
-- 78
appended
79
-- 80
appended
81
-- 82
appended
83
-- 84
appended
85
-- 86
appended
87
-- 88
appended
89
-- 90
appended
91
-- 92
appended
93
-- 94
appended
95
-- 96
appended
97
-- 98
appended
99
-- 100
appended
101
-- 102
appended


In [19]:
display(final_wifo_frame.head())
print ('Size final_wifo_frame: ',len(final_wifo_frame))

,module_code,module_name,property,property_value
0,CS 500,Advanced Software Engineering,Form of module,Lectures and accompanying tutorials
1,CS 500,Advanced Software Engineering,Type of module,Computer Science Fundamental
2,CS 500,Advanced Software Engineering,Level,Master
3,CS 500,Advanced Software Engineering,ECTS,6
4,CS 500,Advanced Software Engineering,NaN,Hours per semester present: 56 h (4 SWS)


Size final_wifo_frame:  2106


### 2.4 Extraction of Courses Metadata from MMDS Catalog

#### Tables that are the start of courses metadata
Create list with tables that are the start of courses tables

In [20]:
# List of table indexes that are the start of a metadata tables of courses 
course_idx = []
for idx in range(len(df_mmds)):
    if bool(re.search('^[a-zA-Z]{1,3} ',df_mmds[idx].columns[0])):
        course_idx.append(idx)
print('List of table indexes of MMDS that are courses: (%s)\n'%len(course_idx), course_idx)

List of table indexes of MMDS that are courses: (8)
 [2, 4, 6, 9, 12, 13, 18, 21]


Fix format of tables with more than 2 columns

In [21]:
# List of tables with more than 2 columns ('Unnamed')
three_column_courses = []
for item in course_idx:
    if len(df_mmds[item].columns)>2:
        three_column_courses.append(item)
print('List of table indexes with more than 2 columns: \n', three_column_courses)


# Dropping extra columns from the tables that are in three_column_courses
drop_columns = ['Unnamed: 0','Unnamed: 1','Unnamed: 2']
for idx in three_column_courses:
    df_mmds[idx] = df_mmds[idx].loc[:,~df_mmds[idx].columns.isin(drop_columns)].copy()


List of table indexes with more than 2 columns: 
 [2, 4]


Check tables with just 1 column

In [22]:
# List of tables with just 1 column
one_column_courses = []
for item in course_idx:
    if len(df_mmds[item].columns)==1:
        one_column_courses.append(item)
print('List of table indexes with just 1 column: \n', one_column_courses)

List of table indexes with just 1 column: 
 []


Final list with tables that are the start of courses tables

In [23]:
print('List of table indexes of MMDS that are courses: (%s)\n'%len(course_idx), course_idx)

List of table indexes of MMDS that are courses: (8)
 [2, 4, 6, 9, 12, 13, 18, 21]


#### Tables that are not the start of courses metadata
Create list with tables that are not the start of courses tables

In [24]:
# List of table indexes that are not the start of a metadata tables of courses 
non_course_idx = []
for idx in range(len(df_mmds)):
    if not bool(re.search('^[a-zA-Z]{1,3} ',df_mmds[idx].columns[0])):
        non_course_idx.append(idx)
print('List of table indexes of MMDS that are not courses: (%s)\n'%len(non_course_idx), non_course_idx)

List of table indexes of MMDS that are not courses: (18)
 [0, 1, 3, 5, 7, 8, 10, 11, 14, 15, 16, 17, 19, 20, 22, 23, 24, 25]


Fix format of tables with more than 2 columns, and delete tables without courses' detail

In [25]:
# List of tables with more than 2 columns ('Unnamed')
three_column_non_courses = []
for item in non_course_idx:
    if len(df_mmds[item].columns)>2:
        three_column_non_courses.append(item)
print('List of table indexes with more than 2 columns: \n', three_column_non_courses)


List of table indexes with more than 2 columns: 
 [0, 1, 8, 10, 11, 15, 16, 17, 20, 22]


In [26]:
# Handling the data frames with more thant 3 columns
remove_non_course_idx = [0,1,8,10,15,20] # remove 0,1,8,10,15,20 indexes
for i in remove_non_course_idx:
    three_column_non_courses.remove(i)
    non_course_idx.remove(i)
    
for idx in three_column_non_courses:
    df_mmds[idx] = df_mmds[idx].loc[:,df_mmds[idx].columns[0:2]].copy()

Check tables with just 1 column

In [27]:
# List of tables with just 1 column
one_column_non_courses = []
for item in non_course_idx:
    if len(df_mmds[item].columns)==1:
        one_column_non_courses.append(item)
print('List of table indexes with just 1 column:\n', one_column_non_courses)


List of table indexes with just 1 column:
 []


Final list with tables that are not the start of courses tables

In [28]:
print('List of table indexes of MMDS that are not courses: (%s)\n'%len(non_course_idx), non_course_idx)

List of table indexes of MMDS that are not courses: (12)
 [3, 5, 7, 11, 14, 16, 17, 19, 22, 23, 24, 25]


### 2.5 Build the Metadata Dataset from MMDS Courses from MMDS Catalog

Join the frames that contain metadata about the MMDS courses

In [29]:
#Desired output with data frame
final_columns = ['module_code','module_name','property','property_value']
final_mmds_frame = pd.DataFrame(columns=final_columns)

#1st loop to append the consider course frames
for idx in range(len(course_idx)):
    course_frame = df_mmds[course_idx[idx]].copy()
    course_name = df_mmds[course_idx[idx]].columns[1]
    course_code = df_mmds[course_idx[idx]].columns[0]
    course_frame.columns = final_columns[2:]
    print(course_idx[idx])
    #if not last course
    if (idx+1)<len(course_idx):
        #2nd for loop to append the non course frames
        for nc_idx in non_course_idx:
            if (nc_idx>course_idx[idx]) & (nc_idx<course_idx[idx+1]):
                intermediate_frame = df_mmds[nc_idx].copy()
                intermediate_frame.columns = final_columns[2:]
                course_frame = pd.concat([course_frame,intermediate_frame],ignore_index = True)
                print('--',nc_idx)
    
    #for last course    
    else:
        for nc_idx in non_course_idx:
            if (nc_idx>course_idx[idx]):
                intermediate_frame = df_mmds[nc_idx].copy()
                intermediate_frame.columns = final_columns[2:]
                course_frame = pd.concat([course_frame,intermediate_frame],ignore_index = True)
                print('--',nc_idx)
    course_frame['module_code'] = course_code
    course_frame['module_name'] = course_name
    final_mmds_frame = pd.concat([final_mmds_frame,course_frame])
    print('appended')

2
-- 3
appended
4
-- 5
appended
6
-- 7
appended
9
-- 11
appended
12
appended
13
-- 14
-- 16
-- 17
appended
18
-- 19
appended
21
-- 22
-- 23
-- 24
-- 25
appended


In [30]:
display(final_mmds_frame.head())
print ('Size final_wifo_frame: ',len(final_mmds_frame))

,module_code,module_name,property,property_value
0,CS 450,Programming Course,Form of module,Lecture and accompanying practical sessions
1,CS 450,Programming Course,Type of module,Computer Science Fundamental
2,CS 450,Programming Course,Level,Master
3,CS 450,Programming Course,ECTS,6
4,CS 450,Programming Course,NaN,Hours per semester present: 84h (6 SWS)


Size final_wifo_frame:  392


### 2.5 Combine MMDS Courses from MMDS Catalog and BI Catalog

Construct a df with all the triples necessary for the KG proposed.

In [31]:
final_frame = pd.concat([final_wifo_frame,final_mmds_frame])
print ('Size final_frame: ',len(final_frame))

# Keep just courses with code
mask_mmds_courses = final_frame.module_code.isin(list(courses_.module_code))
mmds_course_data = final_frame[mask_mmds_courses].copy()

# Fill nan of property
mmds_course_data['property'].ffill(axis=0, inplace=True)

# Dropping duplicate records
mmds_course_data.drop_duplicates(inplace=True)
print ('Size mmds_course_data: ',len(mmds_course_data))

# Grouping the property value
mmds_course_data.property_value = mmds_course_data.property_value.astype(str)
mmds_course_data_grouped = mmds_course_data.groupby(['module_code','module_name','property'])['property_value'].apply(lambda x: ','.join(x)).reset_index().copy()

#removing '\r' from the data
mask_r = mmds_course_data_grouped['property'].str.contains('\r')
features = list(mmds_course_data_grouped.columns)
for col in features:
    mask_r = mmds_course_data_grouped[col].str.contains('\r')
    mmds_course_data_grouped.loc[mask_r,col] = mmds_course_data_grouped.loc[mask_r,col].str.replace('\r',' ')
mmds_course_data_grouped[mask_r]
print ('Size mmds_course_data_grouped: ',len(mmds_course_data_grouped))


Size final_frame:  2498
Size mmds_course_data:  1502
Size mmds_course_data_grouped:  737


## 3. Save MMDS_Courses_Data

In [32]:
# Save file
mmds_course_data_grouped.to_csv('../data/processed/mmds_courses_data.csv',index=False)